In [1]:
import os 
os.chdir('../..')
os.getcwd()

'e:\\Data Driven mkt\\Midterm 2'

In [2]:
# !pip install google-genai

In [ ]:
import numpy as np 
import pandas as pd
from src.llm import RotateGemini

In [5]:
data = pd.read_csv('data/processsed/reviews_cleaned.csv')

C:\Users\nn200\AppData\Local\Temp\ipykernel_10100\4199503533.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/processsed/reviews_cleaned.csv')


In [6]:
data.shape

(1094411, 19)

In [37]:
# Đảm bảo cột 'submission_time' là kiểu datetime
data['submission_time'] = pd.to_datetime(data['submission_time'], errors='coerce')

# Đếm số lượng giá trị theo năm
year_counts = data['submission_time'].dt.year.value_counts()

# Hiển thị kết quả
print(year_counts)

submission_time
2020    215449
2021    202012
2022    192227
2019    143860
2018     97996
2017     54592
2023     49531
2016     33137
2015     21590
2014     18224
2013     15621
2010     13485
2011     12417
2012     11800
2009      9709
2008      2761
Name: count, dtype: int64


In [38]:
data[data['submission_time'].dt.year == 2008]

,review_id,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd,sentiment
24746,24746,1052025473,5,NaN,NaN,0,0,0,2008-12-30,another great fresh product its very gentle an...,"Cleansing, yet gentle",light,NaN,dry,NaN,P7880,Soy Hydrating Gentle Face Cleanser,fresh,39.0,None
24747,24747,1525362153,5,NaN,1.000000,7,0,7,2008-12-26,i bought the trial size because i was nervous ...,LOVE IT!,light,NaN,oily,NaN,P7880,Soy Hydrating Gentle Face Cleanser,fresh,39.0,None
24748,24748,1116517359,5,NaN,0.857143,7,1,6,2008-12-26,i was first introduced to this product at a fr...,My Favorite for 2008,light,NaN,combination,NaN,P7880,Soy Hydrating Gentle Face Cleanser,fresh,39.0,None
24749,24749,1331512522,5,NaN,NaN,0,0,0,2008-12-26,a fresh soothing creamy makeup remover that ge...,Soy Face Cleanser,mediumTan,NaN,oily,NaN,P7880,Soy Hydrating Gentle Face Cleanser,fresh,39.0,None
24750,24750,1362872055,3,NaN,0.200000,5,4,1,2008-12-22,this is a nice product that can remove makeup ...,Nice but not the best,fair,NaN,combination,NaN,P7880,Soy Hydrating Gentle Face Cleanser,fresh,39.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082451,1082451,1391674619,5,NaN,NaN,0,0,0,2008-12-09,i stopped using toners a while ago because the...,Feels good and makes my face look great,lightMedium,NaN,combination,NaN,P217931,Tonique Radiance Clarifying Refining Toner,Lancôme,31.0,None
1082452,1082452,1393130001,2,NaN,NaN,0,0,0,2008-11-04,was reommended this by a sales associate the a...,NaN,NaN,NaN,NaN,NaN,P217931,Tonique Radiance Clarifying Refining Toner,Lancôme,31.0,None
1082453,1082453,923611082,5,NaN,1.000000,1,0,1,2008-10-10,after only 4 applications after washing with t...,Does what it promises,light,NaN,combination,NaN,P217931,Tonique Radiance Clarifying Refining Toner,Lancôme,31.0,None
1082454,1082454,970897744,5,NaN,1.000000,2,0,2,2008-09-18,this toner makes my skin luminous it evened ou...,"Glow Baby, Glow!",tan,NaN,normal,NaN,P217931,Tonique Radiance Clarifying Refining Toner,Lancôme,31.0,None


Số lượng mẫu sau khi lấy: 109426
   review_id   author_id  rating  is_recommended  helpfulness  \
0     891454  1117870798       5             NaN     0.285714   
1      24872  1050064807       4             NaN          NaN   
2     106723   949268502       5             NaN     0.750000   
3      24870  1221497549       4             NaN          NaN   
4      24871  1127277577       5             NaN          NaN   

   total_feedback_count  total_neg_feedback_count  total_pos_feedback_count  \
0                    21                        15                         6   
1                     0                         0                         0   
2                     4                         1                         3   
3                     0                         0                         0   
4                     0                         0                         0   

  submission_time                                        review_text  ...  \
0      2008-08-29  it is

C:\Users\nn200\AppData\Local\Temp\ipykernel_10100\1624422330.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_data = grouped.apply(lambda x: x.sample(frac=0.1, random_state=42) if len(x) > 1 else x)


In [39]:
import json
import re
from typing import List, Optional
import pandas as pd

def _strip_fences(s: str) -> str:
    text = s.strip()
    if text.startswith("```"):
        lines = s.splitlines()
        # bỏ dòng fence đầu
        if lines[0].startswith("```"):
            lines = lines[1:]
        # bỏ dòng fence cuối
        if lines and lines[-1].startswith("```"):
            lines = lines[:-1]
        s = "\n".join(lines)
    # nếu còn marker "json" ở đầu
    if s.strip().lower().startswith("json"):
        s = re.sub(r"^json\s*", "", s, flags=re.IGNORECASE)
    return s.strip()

def get_sentiments(texts: List[Optional[str]], llm_function) -> List[Optional[str]]:
    # Chuẩn bị prompt liệt kê các review (đánh số)
    items = []
    for i, text in enumerate(texts):
        if text is None or text == '' or not text.strip():
            items.append(f"{i+1}. <EMPTY>")
        else:
            items.append(f"{i+1}. {text.strip()}")
    
    items = "\n".join(items)

    prompt = f"""
Below are {len(texts)} review texts (empty ones marked as <EMPTY>).
Extract the sentiment for each review and return a pure JSON array with EXACTLY {len(texts)} elements,
where each element is one of: "positive", "negative", "neutral", or null.

For empty reviews (marked as <EMPTY>), please return null in the JSON array.

IMPORTANT: Your response MUST be a valid JSON array with EXACTLY {len(texts)} elements - no more, no less.

List of reviews:
{items}

Return only valid JSON, no extra text, no markdown fences.
""".strip()

    # Gọi LLM với hàm được truyền vào
    resp = llm_function(prompt)  # resp phải trả về một string

    # Bỏ code fences / marker và cả trailing commas
    raw = _strip_fences(resp)
    raw = re.sub(r",\s*([\]\}])", r"\1", raw)

    # Parse JSON
    try:
        arr = json.loads(raw)
        
        # Kiểm tra nếu không phải là list
        if not isinstance(arr, list):
            raise ValueError(f"Kết quả từ LLM không phải là list: {type(arr)}")
            
        # Xử lý trường hợp độ dài không khớp
        if len(arr) != len(texts):
            print(f"Cảnh báo: Độ dài kết quả ({len(arr)}) không khớp với đầu vào ({len(texts)})")
            
            # Nếu thiếu phần tử, thêm None vào cuối
            if len(arr) < len(texts):
                arr.extend([None] * (len(texts) - len(arr)))
            # Nếu thừa phần tử, cắt bớt
            elif len(arr) > len(texts):
                arr = arr[:len(texts)]
    except json.JSONDecodeError as e:
        raise RuntimeError(f"Không parse được JSON từ LLM:\n{raw}") from e

    # Chuẩn hoá null/none → None
    def norm(x):
        if x is None: 
            return None
        x = str(x).strip().lower()
        return None if x in ("", "null", "none") else x

    # Đảm bảo độ dài mảng kết quả khớp với đầu vào
    result = [norm(x) for x in arr]
    
    return result

def analyze_sentiment_batch(data, llm_function, chunk_size=100, text_column='review_text', sentiment_column='sentiment'):
    result_data = data.copy()
    
    # Khởi tạo cột sentiment nếu chưa có
    if sentiment_column not in result_data.columns:
        result_data[sentiment_column] = None
    
    # Xử lý theo batch
    for start in range(0, len(result_data), chunk_size):
        end = min(start + chunk_size, len(result_data))
        texts = result_data[text_column].iloc[start:end].tolist()
        
        # Phân tích sentiment cho batch hiện tại
        try:
            chunk_sentiments = get_sentiments(texts, llm_function)
            
            # Lấy chính xác các label của chunk
            idx = result_data.index[start:end]
            result_data.loc[idx, sentiment_column] = chunk_sentiments
            
            # In thông tin tiến độ
            print(f"Đã xử lý {end}/{len(result_data)} mẫu")
        except Exception as e:
            print(f"Lỗi ở batch {start}:{end}: {str(e)}")
    
    return result_data
def llm_function(prompt):
    message = [
    {
        "role":"user",
        "content":prompt
    }
    ]
    return llm(message)
df = data[data['submission_time'].dt.year == 2008]
result = analyze_sentiment_batch(data, llm, chunk_size=100)

2025-05-04 01:10:14,212 - INFO - AFC is enabled with max remote calls: 10.


gemini-2.0-flash


2025-05-04 01:10:18,522 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1alpha/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-05-04 01:10:18,529 - INFO - AFC remote call 1 is done.
2025-05-04 01:10:18,531 - INFO - Completion time of gemini-2.0-flash: 4.319777250289917s
2025-05-04 01:10:18,556 - INFO - AFC is enabled with max remote calls: 10.


cache_tokens_details=None cached_content_token_count=None candidates_token_count=498 candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=498)] prompt_token_count=4559 prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=4559)] thoughts_token_count=None tool_use_prompt_token_count=None tool_use_prompt_tokens_details=None total_token_count=5057 traffic_type=None
Cảnh báo: Độ dài kết quả (98) không khớp với đầu vào (100)
Đã xử lý 100/1094411 mẫu
gemini-2.0-flash


2025-05-04 01:10:22,884 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1alpha/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-05-04 01:10:22,892 - INFO - AFC remote call 1 is done.
2025-05-04 01:10:22,894 - INFO - Completion time of gemini-2.0-flash: 4.338430643081665s
2025-05-04 01:10:22,919 - INFO - AFC is enabled with max remote calls: 10.


...

2025-05-04 01:10:23,529 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1alpha/models/gemini-2.0-flash:generateContent "HTTP/1.1 400 Bad Request"
2025-05-04 01:10:23,533 - ERROR - Error: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key expired. Please renew the API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key expired. Please renew the API key.'}]}}
2025-05-04 01:10:23,537 - INFO - AFC is enabled with max remote calls: 10.


Lỗi ở batch 200:300: Không parse được JSON từ LLM:

gemini-2.0-flash


2025-05-04 01:10:27,740 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1alpha/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-05-04 01:10:27,747 - INFO - AFC remote call 1 is done.
2025-05-04 01:10:27,751 - INFO - Completion time of gemini-2.0-flash: 4.213744640350342s
2025-05-04 01:10:27,775 - INFO - AFC is enabled with max remote calls: 10.


cache_tokens_details=None cached_content_token_count=None candidates_token_count=503 candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=503)] prompt_token_count=5126 prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=5126)] thoughts_token_count=None tool_use_prompt_token_count=None tool_use_prompt_tokens_details=None total_token_count=5629 traffic_type=None
Cảnh báo: Độ dài kết quả (99) không khớp với đầu vào (100)
Đã xử lý 400/1094411 mẫu
gemini-2.0-flash


KeyboardInterrupt: 